In [301]:
import dash
from dash import dcc, html, Dash, Input, Output
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
import json

# Load your shapefile and apartment listing data
rent_path = '/Users/Maciek/Desktop/Mieszkania/scrape_listings/otodom_listings/scrape_listings/data/clean/rent_listings_clean_2023-10-14.csv'
sale_path = '/Users/Maciek/Desktop/Mieszkania/scrape_listings/otodom_listings/scrape_listings/data/clean/listings_clean_2023-10-14.csv'
geojson_path = '/Users/Maciek/Desktop/Mieszkania/scrape_listings/otodom_listings/dashboard_listings/data/dzielnice_Warszawy/wwa-geo.json'
df_rent = pd.read_csv(rent_path)
df_sale = pd.read_csv(sale_path)
geojson = json.load(open(geojson_path))

In [302]:
# #CONVERT SHAPEFILE TO GEOJSON
# gdf = gpd.read_file('/Users/Maciek/Desktop/Mieszkania/scrape_listings/otodom_listings/dashboard_listings/data/dzielnice_Warszawy/dzielnice_Warszawy.shp')
# gdf.to_crs(epsg=4326, inplace=True)
# gdf.to_file('/Users/Maciek/Desktop/Mieszkania/scrape_listings/otodom_listings/dashboard_listings/data/dzielnice_Warszawy/wwa-geo.json', driver = 'GeoJSON')
# with open('/Users/Maciek/Desktop/Mieszkania/scrape_listings/otodom_listings/dashboard_listings/data/dzielnice_Warszawy/wwa-geo.json') as geofile:
#     jdataNo = json.load(geofile)    

In [303]:
#######
# DATA PREP
# drop rows with no price
df_sale = df_sale[df_sale['price'].notnull()]

# strip whitespaces from district names
df_sale['district'] = df_sale['district'].str.strip()
df_rent['district'] = df_rent['district'].str.strip()

# delete obscenely high rents (mistakes)
df_rent = df_rent[df_rent['rent']<50000].reset_index(drop=True)
# group rent listings to obtain average rent per meter in district/nbrhood
rent_district_group = df_rent.groupby('district').mean()
rent_nbrhood_group = df_rent.groupby('nbrhood').mean()
rent_per_meter_grouping_district = rent_district_group['rent']/rent_district_group['area_x']
rent_per_meter_grouping_nbrhood = rent_nbrhood_group['rent']/rent_nbrhood_group['area_x']
rent_per_meter_grouping_district = rent_per_meter_grouping_district.reset_index().rename({0: "average_rent_per_sq_meter_district"}, axis=1)
rent_per_meter_grouping_nbrhood = rent_per_meter_grouping_nbrhood.reset_index().rename({0: "average_rent_per_sq_meter_nbrhood"}, axis=1)

df_sale = df_sale.merge(rent_per_meter_grouping_district, on='district', how='left')
df_sale = df_sale.merge(rent_per_meter_grouping_nbrhood, on='nbrhood', how='left')


In [304]:
# create id in geojson to match with district names in df
for i in range(len(geojson['features'])):
    geojson['features'][i]['id'] = i

In [305]:
# create a dictionary with district names and ids from geojson
districts = {}
for i in geojson['features']:
    districts[i['properties']['nazwa_dzie']] = i['id']

In [306]:
# map district ids to names in df_sale
df_sale['district_id'] = df_sale['district'].map(districts)
df_sale['district_id'] = df_sale['district_id'].fillna(df_sale['district_id'].max()+1).astype(int)

In [307]:
# calculate potential broker fee for each listing
def get_broker_fee(adv_type):
    if adv_type in ['biuro nieruchomości', 'no data']:
        return 0.025
    else:
        return 0
    
df_sale['broker_fee'] = df_sale['advertiser_type'].apply(get_broker_fee).astype(float)

In [308]:
# calculate ROI
total_investment = df_sale['price'] + df_sale['broker_fee']*df_sale['price']
annual_rent = df_sale['average_rent_per_sq_meter_district']*df_sale['area']*12
df_sale['ROI'] = annual_rent/total_investment

# calculate price per sq meter
df_sale['price_per_sq_meter'] = df_sale['price']/df_sale['area']


In [321]:
grouping = (df_sale
            .groupby('district')
            .mean()
            .reset_index()
            .dropna()
            .drop(['rooms_No', 'district_id', 'area', 'broker_fee', 'average_rent_per_sq_meter_nbrhood'], axis=1)
            .round(2)
            .sort_index()
)

In [322]:
grouping

,district,price,rooms_No,area,average_rent_per_sq_meter_district,average_rent_per_sq_meter_nbrhood,district_id,broker_fee,ROI,price_per_sq_meter
0,Bemowo,9.486747e+05,2.793187,63.292096,72.847857,71.129186,17.0,0.000000,0.058661,15367.078936
1,Białołęka,6.618829e+05,2.740873,57.117512,67.382318,65.546309,16.0,0.001135,0.069819,11842.493601
2,Bielany,9.584265e+05,2.633570,59.340502,76.592107,76.245746,15.0,0.002246,0.060956,16055.352818
3,Błonia Wilanowskie,1.350000e+06,3.000000,NaN,NaN,NaN,18.0,0.000000,NaN,NaN
4,Chomiczówka,7.790000e+05,3.000000,NaN,NaN,NaN,18.0,0.000000,NaN,NaN
5,Józefów,9.650000e+05,3.000000,89.000000,NaN,NaN,18.0,0.000000,NaN,10842.696629
6,Kawęczyn-Wygoda,7.882750e+05,4.000000,70.000000,NaN,NaN,18.0,0.000000,NaN,11261.071429
7,Konstancin-Jeziorna,3.300000e+06,5.000000,NaN,NaN,NaN,18.0,0.000000,NaN,NaN
8,Marki,8.350000e+05,4.000000,NaN,50.000000,NaN,18.0,0.000000,NaN,NaN
9,Michałowice,6.095000e+05,2.000000,50.000000,40.000000,40.000000,18.0,0.000000,0.038772,12380.000000
